In this notebook we'll try to analyze, what we have in the competition by the simplest possible methods. Nevertheless, it could help to understand the data and split it into some subclasses for futher deeper analysis.

Let's start with importing nessesary libraries.

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

import h5py

from scipy import stats

# 1) Load and concat the labels

Here we load the available labels and put train and test data together. It will be more comfortable to work on both datasets simultaniously. We will differ them by the label: in test dataset we have `target`=0.5, and in the train it is 0 or 1.

`dataset_file_mask` will be used for easy extraction of file path based on its `id`

In [3]:
df1 = pd.read_csv('../input/g2net-detecting-continuous-gravitational-waves/sample_submission.csv')
df2 = pd.read_csv('../input/g2net-detecting-continuous-gravitational-waves/train_labels.csv')

df = pd.concat((df1, df2[df2.target!=-1]), ignore_index=True)

dataset_file_mask = ('../input/g2net-detecting-continuous-gravitational-waves/test/%s.hdf5',
                     '../input/g2net-detecting-continuous-gravitational-waves/train/%s.hdf5')

In [4]:
df.head()

,id,target
0,00054c878,0.5
1,0007285a3,0.5
2,00076c5a6,0.5
3,001349290,0.5
4,001a52e92,0.5


# 2) Load the hdf5 file
The function below loads the data and also performs some moving average (MA) to reduce the noise

In [5]:
def get_MA_SFT(n, MA=12):
    # Load `id` and `target` from dataframe
    id = df.iloc[n]['id']
    target = df.iloc[n]['target']

    # Get the hdf5 file name
    fname = dataset_file_mask[int(target!=0.5)] % id

    # Read the data from the file
    data = h5py.File(fname, 'r')[id]
    print(data)
    
    freqs = np.array(data['frequency_Hz'])
    print(freqs)
    ma_sft = {}
    for detector in ['L1', 'H1']:
        # Get the SFT values
        sft = np.array(data[detector]['SFTs'])*1e22

        # Transform them into power spectrum values
        power = np.power(np.abs(sft), 2)

        # Now calculate the moving average in every `MA` time points
        # For this we get the `cumsum` along the time axis and substract for every `MA` time points
        # The last block may not contain `MA` points, and it is ignored
        power_ma_cumsum = np.cumsum(power, axis=1)
        power_ma_cumsum_0 = np.concatenate((np.zeros((power.shape[0],1)), power_ma_cumsum), axis=1)[:,::MA]
        power_ma = np.diff(power_ma_cumsum_0, axis=1)[:,:-1]/MA
        ma_sft[detector] = power_ma
        
    return ma_sft, freqs

In [10]:
get_MA_SFT(630, MA=12)

<HDF5 group "/15928c672" (3 members)>
[203.83888889 203.83944444 203.84       203.84055556 203.84111111
 203.84166667 203.84222222 203.84277778 203.84333333 203.84388889
 203.84444444 203.845      203.84555556 203.84611111 203.84666667
 203.84722222 203.84777778 203.84833333 203.84888889 203.84944444
 203.85       203.85055556 203.85111111 203.85166667 203.85222222
 203.85277778 203.85333333 203.85388889 203.85444444 203.855
 203.85555556 203.85611111 203.85666667 203.85722222 203.85777778
 203.85833333 203.85888889 203.85944444 203.86       203.86055556
 203.86111111 203.86166667 203.86222222 203.86277778 203.86333333
 203.86388889 203.86444444 203.865      203.86555556 203.86611111
 203.86666667 203.86722222 203.86777778 203.86833333 203.86888889
 203.86944444 203.87       203.87055556 203.87111111 203.87166667
 203.87222222 203.87277778 203.87333333 203.87388889 203.87444444
 203.875      203.87555556 203.87611111 203.87666667 203.87722222
 203.87777778 203.87833333 203.87888889 203

({'L1': array([[1.44097964, 2.32127253, 2.38413048, ..., 2.72460938, 2.07535807,
          3.34472656],
         [2.84438451, 2.2676274 , 3.12703927, ..., 1.50528971, 2.70800781,
          2.47452799],
         [1.52316984, 2.05447801, 1.46935495, ..., 1.56103516, 1.58487956,
          2.97835286],
         ...,
         [1.742788  , 2.20999622, 1.48298009, ..., 1.84204102, 2.71232096,
          1.36735026],
         [2.48153369, 3.61834908, 2.108181  , ..., 1.17936198, 1.93326823,
          2.85506185],
         [2.21614567, 1.63895337, 1.98534362, ..., 1.8745931 , 2.04752604,
          1.13932292]]),
  'H1': array([[1.88811175, 1.88968404, 1.14298471, ..., 2.51204427, 2.38728841,
          2.17797852],
         [1.49417591, 2.83498319, 2.88209184, ..., 1.77547201, 2.65917969,
          1.90576172],
         [1.82154989, 2.29646126, 1.46236006, ..., 1.7672526 , 2.7023112 ,
          1.4132487 ],
         ...,
         [3.75857131, 2.4831953 , 1.70362282, ..., 2.94580078, 2.27986654,
 

# 3) Testing some of the records

The idea of the testing is to plot the average values of SFT with respect both to the frequency or to the time axis. 
Then we do the statistical test, whether or not the achieved values satisfy the normal distribution.

Our null hypothesis will be 'the record is a white gaussian noise'. We calculate the corresponding p-values. If they are suffisiently low, we reject the null hypothesis and say, that the signal is not gaussian. So we must pay attention to it.

In [ ]:
def test_record_view(id):
    # SFT with MA
    ma_sft, freqs = get_MA_SFT(id)

    fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(18,10))
    ax_names = ('Interval No', 'Frequency, Hz')
    for col, detector in enumerate(['L1', 'H1']):
        # Prepare the axes
        axs[0, col].set_title(f'Detector={detector}')
        axs[0, col].set_xlabel(ax_names[0])
        axs[0, col].set_ylabel(ax_names[1])
        axs[0, col].pcolormesh(range(ma_sft[detector].shape[1]), freqs, ma_sft[detector])

        for row in range(2):
            # Calculate mean value for each frequency or for each time interval
            x = np.mean(ma_sft[detector], axis=row)
            k2, p = stats.normaltest(x)

            axs[row+1, col].set_title(f'Detector={detector}; p-value={p:.3}')
            axs[row+1, col].set_xlabel(ax_names[row])
            if row==0:
                axs[row+1, col].plot(x)
            else:
                axs[row+1, col].plot(freqs, x)
                
    fig.tight_layout(pad=1.5)
    plt.show()

## 3.1) `id`=609

In [ ]:
test_record_view(id=609)

* It can be seen from the upper two graphs, that this record does contain the signal. At the same time the noise is not stationary.
* From the middle two graphs we can also see, that the noise is not stationary.
  P-value is low (2e-28) only for the left graph. For the right it is not so low (0.155), that we can reject the null hypothesis.
* From the bottom graphs we can say, it looks like white noise, and the corresponding p-values (0.287 and 0.731) are high.

Conclusion: **the record has non-gaussian noise background and may or may not contain a signal. We must pay attention to it.**

Of course, we can see the signal on the upper graphs by our eyes, but the used statistical criteria is rather weak for this))

## 3.2) `id`=7850

In [ ]:
test_record_view(id=7850)

* It can be seen from the upper two graphs, that this record does contain the signal (line at the bottom).
* From the middle two graphs we can see, that the noise is likely to be stationary at L1 and non-stationary at H1 (p-value is about 0.006).
* From the bottom graphs we can say, it is completly non-gaussian. The record may contain a signal.

Conclusion: **the record may contain a signal. We must pay attention to it.**

## 3.3) `id`=3283

In [ ]:
test_record_view(id=3283)

* We can't see the signal on the left upper graph. The right upper graph is something strange, because the signal is to strong. Also it doesn't have frequency gradient, so it may be just (electrical?) interference. If we had such strong signal on H1, we must had definitely registered it at L1, too. But this is not the case.
* From the middle left graph we can see, that the noise is not stationary at L1. The p-value is also low fow H1, but we need to exclude the interference first to make a conclusion here.
* From the bottom graphs we can clearly see the interference at H1.

Conclusion: **the record may or may not contain a signal. But it definitely contains interference that should not be confused with it.**

## 3.4) `id`=8100

In [ ]:
test_record_view(id=8100)

This is the record from the train set. 
All the graphs say the same thing: **the record looks like white noise, and me have to investigate it further**.

# 4) Process all records

Now we do the same, but with all records and without plots. Then we'll compare the results for train and test sets.

In [ ]:
def test_record(id):
    ma_sft, freqs = get_MA_SFT(id)

    results = {'freq' : freqs[0]}
    
    for detector in ['L1', 'H1']:
        results[detector] = []
        for row in range(2):
            # Calculate mean value for each frequency or for each time interval
            x = np.mean(ma_sft[detector], axis=row)
            k2, p = stats.normaltest(x)
            results[detector].append(p)
            
    return [results['freq']] + results['L1'] + results['H1']

In [ ]:
# Add zero-filled columns to our dataframe
columns = ['freq', 'L1_time', 'L1_freq', 'H1_time', 'H1_freq']
df[columns] = 0

# Now calculate p-values and put it inside the dataframe
for id in range(len(df)):
    if id % 100 == 0: 
        print(f'Processing #{id}')
    df.loc[id, columns] = test_record(id)
    
df.head()

# 5) Finally, let's display some statistics

We introduse some `treshold` and a set of criteria, when we reject the particulra null hypothesis. All is put into one function

In [ ]:
def print_criteria_statistics(ds, treshold=0.01):
    # A set of criteria for every column
    criteria = [df[col]<treshold for col in columns[1:]]
    
    # A dict of criteria for datasets
    criteria_ds = {'train' : df.target != 0.5, 'test': df.target == 0.5}

    # Total length
    total = len(df[criteria_ds[ds]])

    print(f'Dataset = {ds}; length = {total}')
    print('-'*80)

    for n, col in enumerate(columns[1:]):
        l = len(df[criteria_ds[ds] & criteria[n]])
        print(f'{col}  is not gauss for\t{l}/{total},\twhich is {100*l/total:.3}%')

    print('-'*80)

    l = len(df[criteria_ds[ds] & criteria[0] & criteria[2]])
    print(f'Time L&H is not gauss for\t{l}/{total},\twhich is {100*l/total:.3}%')

    l = len(df[criteria_ds[ds] & criteria[1] & criteria[3]])
    print(f'Freq L&H is not gauss for\t{l}/{total},\twhich is {100*l/total:.3}%')

    print('-'*80)

    l = len(df[criteria_ds[ds] & (criteria[0] | criteria[2])])
    print(f'Time L|H is not gauss for\t{l}/{total},\twhich is {100*l/total:.3}%')

    l = len(df[criteria_ds[ds] & (criteria[1] | criteria[3])])
    print(f'Freq L|H is not gauss for\t{l}/{total},\twhich is {100*l/total:.3}%')

Now, display the results

In [ ]:
print_criteria_statistics(ds = 'train')

print('\n')

print_criteria_statistics(ds = 'test')

It's interesting! So...
* **Stationarity.**

In the train set we never have non stationary noise on both detectors simultaniously, and only in 3.5% we have it on one of the detectors.
In the test set we have it on both detectors in 11.2% and on at least one of them in 21.3%.

* **Frequency "artifacts"**

These artifacts could be potential signals or a glitches, or an interferences.
In the train set we have it in 7% on the both detectors and in 15% on at least one of them. In the test the corresponding values are 1.9% and 8.8%. 

Now let's set even less value for the `treshold` and look, what happens.

In [ ]:
print_criteria_statistics(ds = 'train', treshold=1e-5)

print('\n')

print_criteria_statistics(ds = 'test', treshold=1e-5)

We can see, that now all train signals look stationary (speaking corectly, we don't reject our null hypothesis), and 18.5% of test signals not. But the most interesting is that frequency artifacts are not independent on both detectors. If it was so, than, for test set, we'll have 0.0176*0.0406 = 0.07% of not gauss 'Freq L&H', but we have 1.17%. So, almost all of the records with not gauss 'Freq L&H' (we have selected 93 of them) are likely to contain a signal.

Let's get their exact ids and look at them to see, if it's true.

# 6) Investigating not gaussian 'Freq L&H'

Let's look at, say, first 5 candidates

In [ ]:
treshold = 1e-5

candidates = df[(df.target==0.5) & (df.L1_freq<treshold) & (df.H1_freq<treshold)].index.tolist()

for n in candidates[:5]:
    print(f'Candidate {n}')
    test_record_view(id=n)

The first one (#50) is very interesting! On L1 detector we see the increasing frequency, and on H1 detector it is different and approximately constant. So, we may deal with the signal and the interference at the same time))
When looking carefully we can also see the small peak near 250.1Hz on H1 detector, which may correspond to the signal on L1.

Other 4 records have approximately the same frequencies of peaks on L1 and H1.

You may notice, that if the signal frequency varies a lot, it doesn't affect the average power spectrum much. This signal will be definitely lost with the proposed approach. So, to conclude, there's much work, and we must be very carefull with the signals we have. 